In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time

In [2]:
# 查看當前路徑
# import os
# print (os.getcwd())

In [3]:
# from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections

In [4]:
chrome_options = webdriver.ChromeOptions() # 不打開瀏覽器執行
chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage') #overcome limited resource problems
chromedriver_path = "/Users/leo/Desktop/Analytics/2020 Spring/ISE 533/project1 receipe/chromedriver"

In [5]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [6]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        print("find xpath timeout:",xpath)


In [49]:
all_url = [
#             'https://www.food.com/search/lamb', #lamb
            'https://www.food.com/search/beef', #beef
#             'https://www.food.com/search/chicken', #chicken
#             'https://www.food.com/search/pork', #pork
#             'https://www.food.com/search/turkey', #turkey
#             'https://www.food.com/search/salmon', #salmon
#             'https://www.food.com/search/tuna', #tuna
#             'https://www.food.com/search/shrimp', #shrimp
#             'https://www.food.com/search/Halibut', #Halibut
#             'https://www.food.com/search/crab', #crab
#             'https://www.food.com/search/clam', #clam
#             'https://www.food.com/search/oyster', #oyster
#             'https://www.food.com/search/Vegetarian', #Vegetarian
#             'https://www.food.com/search/egg', #egg
#             'https://www.food.com/search/salad', #salad
#             'https://www.food.com/search/fruit' #fruit
          ]

In [50]:
recipe_xpath = "//div/div[@class='fd-inner-tile']"
title_xpath = "//div[@class='recipe-title']/h1"
time_xpath = "//div[@class='recipe-facts__details recipe-facts__time']/span[2]"
rating_section_xpath = "//div[@class='recipe-details']"
clicktofull_button_xpath = "//a[@class='recipe-layout__show-more-link theme-color']"
ingred_xpath = "//span[@class='recipe-ingredients__ingredient-part'][2]/a[@class='theme-color']"
nutrition_button_xpath = "//a[@class='recipe-ingredients__nutrition-button theme-color']"
nutrition_section_xpath = "//section[@class='recipe-nutrition__info']"
nutrition_exit_path = "//button[@class='recipe-close-button__icon icon-gk-close']"
# calorie_xpath = "//p[@class='recipe-nutrition__item bold']"
# reviews_button_xpath = "//div[@class='theme-color conversation__filter conversation__filter--reviews']"
reviews_more_button_xpath = "//span[@class='conversation__show-more__text']"
reviews_section_xpath = "//div[@class='post']"
reviewer_personal_rating_xpath = "//div[@class='post__rating']/div[@class='stars-rate']/div[@class='stars-rate__filler']"
reviewer_personal_name_xpath = "//a[@class='post__author-link theme-color']"
# reviews_for_scrolling_xpath = "//h3[@class='recipe-suggestions__title']"
price_iframe_xpath = "//div[@id='CCKU-VPLO-TYHX-LNGN']/iframe"
nutrition_close_xpath = "//button[@class='recipe-close-button__icon icon-gk-close']"


In [51]:
all_url = collections.deque(all_url)
total_ep_urls = set()
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:
    while all_url:
        driver.get(all_url.popleft())
        ep_urls = set()
        start = time.perf_counter()
        while 1:
            if (time.perf_counter() - start)>=20:
                break
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            if driver.current_url == 'https://www.food.com/search/lamb?pn=30':   # THIS IS WRONG!!!!!!!!!!!
                break
            time.sleep(0.01)     
            
        allpaths_to_click = driver.wait_elements_xpath(recipe_xpath)
#         allpaths_to_click = wait_to_get(driver, lambda dr: dr.find_elements_by_xpath(recipe_xpath))
        for section in allpaths_to_click:
            if '(' not in section.text:
                continue
            section_list = section.text.split('\n')
#             print(section_list)
            rating_amount = section_list[2].strip('()')
            if 'k' in rating_amount:
                rating_amount = float(rating_amount.strip('k'))*1000
            else:
                rating_amount = float(rating_amount)
#             print(rating_amount)
            if rating_amount>=280:
                link = section.find_element_by_tag_name('a').get_attribute('href')
                print(rating_amount,'  ',link)
                ep_urls.add(link)
            else:
    #             print(rating_amount,'  not enough')
                continue   

            if len(ep_urls)>=250:
    #             print('@@@',len(ep_urls)) 
                break
        total_ep_urls.update(ep_urls)
        print('ep_url:',len(ep_urls),'total_ep_url:',len(total_ep_urls))

356.0    https://www.food.com/recipe/best-beef-stroganoff-73922
335.0    https://www.food.com/recipe/szechuan-noodles-with-spicy-beef-sauce-48760
280.0    https://www.food.com/recipe/beef-patties-in-onion-gravy-37413
289.0    https://www.food.com/recipe/mahogany-beef-stew-80536
ep_url: 4 total_ep_url: 4


In [56]:
class EP_Recipe():
    page_url = None
    title = None
    total_time = None
    rating = None
    ingredient = {}
    nutrition = {}
    personal_rating = {}
    
    
    def __init__(self, page):
        print('attempting to build from: '+page)
        try:
            with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:
                url = page
                url +="?ftab=reviews"
                driver.get(url)
                self.build_recipie(driver)
        except Exception as x:
            print('Could not build from %s, %s'%(page,x))

    def build_recipie(self, driver):
        self.page_url = self.get_url(driver)
        self.title = self.get_title(driver)
        self.total_time = self.get_time(driver)
        self.rating = self.get_rating(driver)
        self.personal_rating = self.get_per_rating(driver)
        self.ingred = self.get_ingred(driver)
        self.nutrition = self.get_nutrition(driver)

    
    def get_url(self, driver):
        return driver.current_url
    
    def get_title(self, driver):
#         return wait_to_get(driver, lambda dr: dr.find_element_by_xpath(title_xpath)).text
        return driver.wait_element_xpath(title_xpath).text
        
    def get_time(self,driver):
        try:
    #             t = wait_to_get(driver, lambda dr: dr.find_element_by_xpath(time_xpath)).text
            t = driver.wait_element_xpath(time_xpath).text
            total_min = 0
            if 'hrs' in t or 'hr' in t: 
                    t = t.replace('h',' h') # add a space for split
            if "mins" in t or 'min' in t:
                t = t.replace('m',' m')
            t = list(t.split())
            for i in range(len(t)-1):
                if "hrs" in t[i+1] or 'hr' in t[i+1]:
                    total_min += float(t[i])*60
                if "mins" in t[i+1] or 'min' in t[i+1]:
                    total_min += float(t[i])
            return total_min
        except:
            return None

    def get_rating(self, driver):
        try:
            rating = driver.wait_element_xpath(rating_section_xpath)\
                                      .find_element_by_class_name('stars-rate__filler')\
                                      .get_attribute('style') 
#             rating = wait_to_get(driver, lambda dr: dr.find_element_by_xpath(rating_section_xpath))\
#                                                   .find_element_by_class_name('stars-rate__filler')\
#                                                   .get_attribute('style') 
            rating1to5 = round(float(rating.strip('width: %;'))/20,3)
            return rating1to5
        except:
            return None

    def get_ingred(self, driver):
        try:
#             wait_to_get(driver, lambda dr: dr.find_element_by_xpath(clicktofull_button_xpath)).click()
            driver.wait_element_xpath(clicktofull_button_xpath).click()
#             ingred_elements = wait_to_get(driver, lambda dr: dr.find_elements_by_xpath(ingred_xpath))
            ingred_elements = driver.wait_elements_xpath(ingred_xpath)
            ingre_list=set()
            for ingred in ingred_elements:
                ingre_list.add(ingred.text)
            return list(ingre_list)
        except:
            return None

    def get_nutrition(self, driver):
        try:
#             driver.find_element_by_xpath(nutrition_button_xpath).click()
            driver.wait_element_xpath(nutrition_button_xpath).click()
#             calorie_text = wait_to_get(driver, lambda dr: dr.find_element_by_xpath(nutrition_section_xpath)).text
            calorie_text = driver.wait_element_xpath(nutrition_section_xpath).text
            n = calorie_text.split('\n')
            nutrition_dict={}
            for i in range(len(n)-1):
                if "Calories:" in n[i]:
                    nutrition_dict['Calories'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
                if "Total Fat" in n[i]:
                    nutrition_dict['Fat'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
                if "Cholesterol" in n[i]:
                    nutrition_dict['Cholesterol'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
                if "Sodium" in n[i]:
                    nutrition_dict['Sodium'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
                if "Carbohydrate" in n[i]:
                    nutrition_dict['Carbohydrate'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
                if "Protein" in n[i]:
                    nutrition_dict['Protein'] = float(re.sub(u"([^\u0030-\u0039])","",n[i]))
#             driver.find_element_by_xpath(nutrition_close_xpath).click()
#             time.sleep(0.5)
            return nutrition_dict
        except:
            return None

    def get_per_rating(self, driver):
        while 1:
#             more_reviews_button = wait_to_get(driver, lambda dr: dr.find_element_by_xpath(reviews_more_button_xpath))
            more_reviews_button = driver.wait_element_xpath(reviews_more_button_xpath)
            if not more_reviews_button:
                break
            try:
                more_reviews_button.click()
            except:
                break
        
        try:
            time.sleep(0.05)
            reviews_dict = {}
#             review_sections = driver.find_elements_by_xpath(reviews_section_xpath)
            review_sections = driver.wait_elements_xpath(reviews_section_xpath)
            review_count = 0
            for personal_review in review_sections:
                try:
                    reviewers_rating_element = personal_review.find_element_by_class_name('stars-rate__filler')     
                except:
                    continue
                reviewers_rating = reviewers_rating_element.get_attribute('style')
                review_count+=1
                personal_rating1to5 = round(float(reviewers_rating.strip('width: %;'))/20,3)
                reviewers_name = personal_review.find_elements_by_tag_name('a')  # find personal name(reviewers_name[1].text)
                reviews_dict[reviewers_name[1].text] = personal_rating1to5

            return reviews_dict
        except Exception as x:
            print('review:',x)
            return None

In [57]:
p = multiprocessing.Pool(2)
output = p.map(EP_Recipe,total_ep_urls)

attempting to build from: https://www.food.com/recipe/best-beef-stroganoff-73922
attempting to build from: https://www.food.com/recipe/mahogany-beef-stew-80536
1 hr 55 mins
35 mins
find xpath timeout: //span[@class='conversation__show-more__text']
find xpath timeout: //span[@class='conversation__show-more__text']
attempting to build from: https://www.food.com/recipe/szechuan-noodles-with-spicy-beef-sauce-48760
30 mins
attempting to build from: https://www.food.com/recipe/beef-patties-in-onion-gravy-37413
35 mins
find xpath timeout: //span[@class='conversation__show-more__text']
find xpath timeout: //span[@class='conversation__show-more__text']


In [58]:
pickle.dump(output,open('epi_recipes.final','wb'))
data = pickle.load(open('epi_recipes.final','rb'))

ar = []
for i in data:
    ar.append(i.__dict__)
# pickle.dump(ar,open('epi_recipe_dict_form.dict','wb'))

with open('recipes_data.json', 'w') as fp:
    json.dump(ar, fp)

In [59]:
df = pd.read_json('recipes_data.json')
df

,page_url,title,total_time,rating,personal_rating,ingred,nutrition
0,https://www.food.com/recipe/best-beef-strogano...,BEST BEEF STROGANOFF,35,4.49,"{'Jimmy801': 5.0, 'shlee111': 5.0, 'AltoRose':...","[garlic clove, ground beef, butter, salt, sour...","{'Calories': 5822.0, 'Fat': 321.0, 'Cholestero..."
1,https://www.food.com/recipe/mahogany-beef-stew...,MAHOGANY BEEF STEW,115,4.64,"{'FrenchBunny': 5.0, 'Lauri': 5.0, 'MommyWhoBa...","[olive oil, onions, thyme, bay leaves, garlic ...","{'Calories': 6563.0, 'Fat': 262.0, 'Cholestero..."
2,https://www.food.com/recipe/szechuan-noodles-w...,SZECHUAN NOODLES WITH SPICY BEEF SAUCE,30,4.52,"{'Deantini': 5.0, 'chia2160': 5.0, 'JustJanS':...","[fresh ginger, onions, cornstarch, ground beef...","{'Calories': 6171.0, 'Fat': 256.0, 'Cholestero..."
3,https://www.food.com/recipe/beef-patties-in-on...,BEEF PATTIES IN ONION GRAVY,35,4.62,"{'MizzNezz': 5.0, 'Donna': 5.0, 'riffraff': 5....","[water, ground beef, egg, Worcestershire sauce...","{'Calories': 3591.0, 'Fat': 19.0, 'Cholesterol..."
